# The Project #
1. This is a project with minimal scaffolding. Expect to use the the discussion forums to gain insights! It’s not cheating to ask others for opinions or perspectives!
2. Be inquisitive, try out new things.
3. Use the previous modules for insights into how to complete the functions! You'll have to combine Pillow, OpenCV, and Pytesseract
4. There are hints provided in Coursera, feel free to explore the hints if needed. Each hint provide progressively more details on how to solve the issue. This project is intended to be comprehensive and difficult if you do it without the hints.

### The Assignment ###
Take a [ZIP file](https://en.wikipedia.org/wiki/Zip_(file_format)) of images and process them, using a [library built into python](https://docs.python.org/3/library/zipfile.html) that you need to learn how to use. A ZIP file takes several different files and compresses them, thus saving space, into one single file. The files in the ZIP file we provide are newspaper images (like you saw in week 3). Your task is to write python code which allows one to search through the images looking for the occurrences of keywords and faces. E.g. if you search for "pizza" it will return a contact sheet of all of the faces which were located on the newspaper page which mentions "pizza". This will test your ability to learn a new ([library](https://docs.python.org/3/library/zipfile.html)), your ability to use OpenCV to detect faces, your ability to use tesseract to do optical character recognition, and your ability to use PIL to composite images together into contact sheets.

Each page of the newspapers is saved as a single PNG image in a file called [images.zip](./readonly/images.zip). These newspapers are in english, and contain a variety of stories, advertisements and images. Note: This file is fairly large (~200 MB) and may take some time to work with, I would encourage you to use [small_img.zip](./readonly/small_img.zip) for testing.

Here's an example of the output expected. Using the [small_img.zip](./readonly/small_img.zip) file, if I search for the string "Christopher" I should see the following image:
![Christopher Search](./readonly/small_project.png)
If I were to use the [images.zip](./readonly/images.zip) file and search for "Mark" I should see the following image (note that there are times when there are no faces on a page, but a word is found!):
![Mark Search](./readonly/large_project.png)

Note: That big file can take some time to process - for me it took nearly ten minutes! Use the small one for testing.

In [4]:
import zipfile
import PIL
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import cv2 as cv
import numpy as np

In [2]:
def face_name_search(file, name):
    face_name_dic = process_images(file)
    name_found_dic = name_search_dictionary_constructor(face_name_dic, name)
    face_sheet_list = face_sheet_constructor(name_found_dic)
    contact_sheet_height = height_calculator(face_sheet_list)
    contact_sheet = contact_sheet_constructor(face_sheet_list, contact_sheet_height)
    return contact_sheet

In [5]:
def face_detector(file):
    face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')
    cv_img=cv.imread(file)
    faces = face_cascade.detectMultiScale(cv_img, scaleFactor = 1.35)
    face_list = []
    for x,y,w,h in faces:
        face = cv_img[y:y+h,x:x+w]
        face_list.append(face)
    return face_list

In [4]:
def process_images(zip_file):
    face_name_dictionary = {}
    zip_ref = zipfile.ZipFile(zip_file)
    for zip_obj in zip_ref.namelist():
        print("Now processing: " + zip_obj)
        file_dictionary = face_name_dictionary[zip_obj] = {}
        file = zip_ref.extract(zip_obj)
        image = file_dictionary['image'] = Image.open(file)
        print("Image for '" + zip_obj + "' now added to dictionary.")
        text = file_dictionary['text'] = pytesseract.image_to_string(image).lower()
        print("Text for '" + zip_obj + "' now added to dictionary.")
        faces = file_dictionary['faces'] = face_detector(file)
        print("Faces for '" + zip_obj + "' now added to dictionary.")
    return face_name_dictionary

In [22]:
def name_search_dictionary_constructor(dictionary, name):
    name_found_dic = {}
    for filename in dictionary.keys():
        if name.lower() in dictionary[filename]['text']:
            name_found_dic[filename] = dictionary[filename]
    return name_found_dic

In [23]:
def face_sheet_constructor(dictionary):
    import math
    face_sheet_list = []
    for filename in dictionary.keys():
        dic = dictionary[filename]
        face_rows = math.ceil(len(dic["faces"]) / 5)
        if face_rows == 0:
            face_sheet_label = PIL.Image.new("RGB", (500, 40), (256,256,256))
            label = 'Results found in file ' + filename
            message = 'But there were no faces in that file!'
            d = ImageDraw.Draw(face_sheet_label)
            font = ImageFont.truetype('readonly/fanwood-webfont.ttf', size=20)
            d.text((0,0), label, font=font, fill=(0,0,0,128))
            d.text((0,20), message, font=font, fill=(0,0,0,128))
        else:
            face_sheet = PIL.Image.new("RGB", (500, face_rows*100), (0,0,0))
            position = (0,0)
            for i in range(len(dic['faces'])):
                resize = cv.resize(dic['faces'][i], (100,100))
                resize_color = cv.cvtColor(resize, cv.COLOR_BGR2RGB)
                resize_img = Image.fromarray(resize_color, "RGB")
                x,y= position
                face_sheet.paste(resize_img, (x,y))
                if x >= 400:
                    y += 100
                    x = 0
                else:
                    x += 100
                position = (x,y)
            face_sheet_label = PIL.Image.new("RGB", (face_sheet.width, face_sheet.height+40), (256,256,256))
            label = 'Results found in file ' + filename
            d = ImageDraw.Draw(face_sheet_label)
            font = ImageFont.truetype('readonly/fanwood-webfont.ttf', size=20)
            d.text((0,0), label, font=font, fill=(0,0,0,128))
            face_sheet_label.paste(face_sheet, (0,20))
        face_sheet_list.append(face_sheet_label)
    return face_sheet_list

In [24]:
def height_calculator(face_sheet_list):
    height_collector = 0
    for sheet in face_sheet_list:
        height_collector += sheet.height
    return height_collector

In [25]:
def contact_sheet_constructor(face_sheet_list, contact_sheet_height):
    contact_sheet = PIL.Image.new("RGB", (500,contact_sheet_height))
    position = 0
    for sheet in face_sheet_list:
        contact_sheet.paste(sheet, (0,position))
        position += sheet.height
    return contact_sheet

In [34]:
large_contact_sheet = face_name_search('readonly/images.zip', "Mark")
display(large_contact_sheet)

In [50]:
small_contact_sheet = face_name_search('readonly/small_img.zip', "Christopher")
display(small_contact_sheet)